In [43]:
import glob
import sys
import subprocess

sys.path.append("/u/lukas.rilling/dev/")

from returnn_training_progress import get_epoch_data
from returnn_training_plot_nb import plot_df
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [46]:
base_dir = "/u/lukas.rilling/experiments/glow_tts_asr_v2/output/"
wer_files = str(
    subprocess.check_output(
        f"find {base_dir} -type l -name 'wer' | grep -E 'dev-other' | grep -E -v 'jaist_project'",
        shell=True,
    ),
    "utf-8",
).split("\n")[:-1]

len(wer_files)

585

In [ ]:
wer_files_first_index = []
experiment_group = []
wers = []
for i,f in enumerate(wer_files):
    with open(f, "r") as ff:
        wers.append(float(ff.readline().replace("\n", "")))
    folders = ["librispeech_glow_asr", "joint_training/default", "joint_training/conformer_coupling", "joint_training/given_alignments"]
    found = False
    for folder in folders:
        if folder in f:
            experiment_group.append(folder)
            found = True
            break
    assert len(wers) == len(experiment_group), f"Can't find this experiment group: {f}"

    possible_splitter = ["search_params", "lm_tuning", "tuning", "tuned"]
    splitter = ""
    for s in possible_splitter:
        if s in f:
            splitter = s
            break
    if splitter == "":
        wer_files_first_index.append(f.replace("default_250/dev-other/sclite/wer", "").replace("search/dev-other/sclite/wer", ""))
        continue
    wer_files_first_index.append(f[:f.index(splitter)])
assert len(wers) == len(wer_files_first_index) and len(wers) == len(experiment_group), f"Well wer has length: {len(wers)} and experiment_group has length {len(experiment_group)}"

In [ ]:
import pandas as pd

index = pd.MultiIndex.from_arrays([experiment_group, wer_files_first_index, wer_files], names=("Group", "Experiment", "Tuning"))

df = pd.DataFrame({"WER (dev-other)": wers}, index=index)

In [ ]:
count_series = df.index.get_level_values("Experiment").value_counts()

df["Count"] = df.index.get_level_values("Experiment").map(count_series)
df["Tuned"] = df["Count"] > 3
df.loc[
        "joint_training/default",
        "/u/lukas.rilling/experiments/glow_tts_asr_v2/output/experiments/librispeech/joint_training/default/raw_audio/glowTTS_ASR_conformer_x_vector_v2/",
]

/tmp/ipykernel_2403159/524401006.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  df.loc[


,WER (dev-other),Count,Tuned
Tuning,,,
/u/lukas.rilling/experiments/glow_tts_asr_v2/output/experiments/librispeech/joint_training/default/raw_audio/glowTTS_ASR_conformer_x_vector_v2/search/dev-other/sclite/wer,114.2,1,False


In [ ]:
df_grouped = df.groupby(level=["Group","Experiment"]).min()

In [ ]:
ctc_scores = []
ctc_dev_scores = []
finished = []
for index, series in df_grouped.iterrows():
    data = get_epoch_data((f"{index[1]}training/work/learning_rates").replace("output", "alias"), None)
    if data is None or "ctc" not in  data[list(data.keys())[-1]]["error"]:
        finished.append(False)
        ctc_scores.append(False)
        ctc_dev_scores.append(False)
    else:
        last_epoch_data = data[list(data.keys())[-1]]
        finished.append(True)
        ctc_scores.append(last_epoch_data["error"]["ctc"])
        ctc_dev_scores.append(last_epoch_data["error"]["dev_loss_ctc"])

df_grouped["CTC"] = ctc_scores
df_grouped["dev CTC"] = ctc_dev_scores
df_grouped["Still running"] = [not x for x in finished]

df_grouped.head(5)

WER (dev-other)  \
Group                             Experiment                                                            
joint_training/conformer_coupling /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...             26.4   
                                  /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...             36.8   
                                  /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...             31.8   
                                  /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...             26.5   
                                  /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...             32.0   

                                                                                      Count  \
Group                             Experiment                                                  
joint_training/conformer_coupling /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...      7   
                                  /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...      7   
                                  /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...      7   
                                  /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...      7   
                                  /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...      7   

                                                                                      Tuned  \
Group                             Experiment                                                  
joint_training/conformer_coupling /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...   True   
                                  /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...   True   
                                  /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...   True   
                                  /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...   True   
                                  /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...   True   

                                                                                           CTC  \
Group                             Experiment                                                     
joint_training/conformer_coupling /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...  0.291952   
                                  /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...  0.204306   
                                  /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...  0.023758   
                                  /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...  0.289078   
                                  /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...  0.023308   

                                                                                       dev CTC  \
Group                             Experiment                                                     
joint_training/conformer_coupling /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...  0.564334   
                                  /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...  0.880877   
                                  /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...   0.86303   
                                  /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...   0.56867   
                                  /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...  0.866149   

                                                                                      Still running  
Group                             Experiment                                                         
joint_training/conformer_coupling /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...          False  
                                  /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...          False  
                                  /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...          False  
                                  /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...          False  
                                

In [ ]:
df_finished = df_grouped[~df_grouped["Still running"]]

df_finished.head(5)

WER (dev-other)  \
Group                             Experiment                                                            
joint_training/conformer_coupling /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...             26.4   
                                  /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...             36.8   
                                  /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...             31.8   
                                  /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...             26.5   
                                  /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...             32.0   

                                                                                      Count  \
Group                             Experiment                                                  
joint_training/conformer_coupling /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...      7   
                                  /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...      7   
                                  /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...      7   
                                  /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...      7   
                                  /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...      7   

                                                                                      Tuned  \
Group                             Experiment                                                  
joint_training/conformer_coupling /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...   True   
                                  /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...   True   
                                  /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...   True   
                                  /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...   True   
                                  /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...   True   

                                                                                           CTC  \
Group                             Experiment                                                     
joint_training/conformer_coupling /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...  0.291952   
                                  /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...  0.204306   
                                  /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...  0.023758   
                                  /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...  0.289078   
                                  /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...  0.023308   

                                                                                       dev CTC  \
Group                             Experiment                                                     
joint_training/conformer_coupling /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...  0.564334   
                                  /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...  0.880877   
                                  /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...   0.86303   
                                  /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...   0.56867   
                                  /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...  0.866149   

                                                                                      Still running  
Group                             Experiment                                                         
joint_training/conformer_coupling /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...          False  
                                  /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...          False  
                                  /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...          False  
                                  /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...          False  
                                